# Imports

In [ ]:
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from scipy.linalg import svd
import sklearn.cluster as cluster
import plotly.express as px
import plotly.graph_objects as go

np.random.seed(1234)

In [ ]:
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
# mpl.rcParams.update({"axes.grid": True})

# 7.1 Linear Regression

Regression - requires a real-valued outcome

Simple example - model someones weight given their age
More complex - use their age, height, and country of origin

What isn't regression?
Predict if a person is a male or female given data about them? Not because output not real-valued (categorical output)

Simple Regression
- Just one predictor variable, e.g. given gross square footage, what is sale price of house?
- Do a scatter plot of sale price vs. living area

Simple Linear Regression
- Model assumes outcome vs. feature is linear

What is lowess?
- Looks like piecewise-linear
- More difficult to explain though

What is a model?
- Idealized representation of a system
- *All models are wrong, but some are useful*

Linear models
- Models are linear when predictions are linear combinations of variables

### Data Load - California Housing Data Set

In [ ]:
# housing = datasets.fetch_california_housing()
# df = pd.DataFrame(data=housing.data, columns=housing.feature_names)
df = pd.read_csv("./data/housing.csv")
display(df)

### Scatter with Lowess Trendline

In [ ]:
px.scatter(
    df,
    x="GrLivArea",
    y="SalePrice",
    trendline="lowess",
    trendline_color_override="black",
)

# 7.2 Tips Dataset

Row is a table that was served by a waiter. Sex is who paid, not sex of waiter.  
Going to concentrate on total bill

In [ ]:
data = sns.load_dataset("tips", cache=False)
data.head()

In [ ]:
px.scatter(
    data,
    x="total_bill",
    y="tip",
    trendline="ols",
    trendline_color_override="black",
)

# 7.3 Creating a Plotly and Scikit-Learn Model

Scikit Process
- Instantiate linear model object
- Fit model
- Use the object to make predictions

### Create Model

In [ ]:
features = data[["total_bill"]]
target = data["tip"]
f = linear_model.LinearRegression(fit_intercept=False)
f.fit(features, target)

### Predictions

In [ ]:
data["prediction"] = f.predict(features)
data.head()

### Plot

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=data["total_bill"], y=data["tip"], mode="markers", name="actual")
)
fig.add_trace(
    go.Scatter(
        x=data["total_bill"], y=data["prediction"], mode="lines", name="prediction"
    )
)
fig.update_layout(font_size=20)

### Model Params


In [ ]:
display([f.coef_, f.intercept_])

### Regresion with Plotly

In [ ]:
px.scatter(
    data,
    x="total_bill",
    y="tip",
    trendline="ols",
    trendline_color_override="red",
)

# 7.4 Defining Loss Functions

Loss Function
- Numerically computes the badness of a model
- Tells us how wrong it is
- Most common is L2 (squared loss), as in L2 norm

Definition of L2 Loss Function: $$L(y,yhat) = (y - yhat)^2$$

Example:
- y = 5.15
- yhat = 1.04
- loss = (5.15 - 1.04)^2 = 16.89

Mean Square Error (MSE)

Compute the average of L2 over entire dataset, mean squared errorr, just take average of all instances of L(y,yhat)

In [ ]:
(5.15 - 1.04) ** 2

# 7.5 Computing L2


In [ ]:
l2_loss = (data["tip"] - data["prediction"]) ** 2
l2_loss.mean()

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(data["tip"], data["prediction"])

# 7.6 Optimizing L2

Different choice of slope variable theta --> different predictions --> different L2

In [ ]:
mean_squared_error(data["tip"], data["total_bill"] * 0.2)

In [ ]:
def mse_given_theta(theta: float):
    return mean_squared_error(data["tip"], data["total_bill"] * theta)

In [ ]:
mse_given_theta(0.2)

## Demo of Range of Objective Variable Evaluations vs. MSE

In [ ]:
thetas = np.linspace(0.1, 0.2, 100)
mses = [mse_given_theta(theta) for theta in thetas]
# display(mses)
fig = px.line(x=thetas, y=mses)
fig.update_layout(xaxis_title="theta", yaxis_title="MSE", font_size=20)

# 7.7 Optimize L2 with Scipy

In [ ]:
import scipy.optimize

In [ ]:
def g(x):
    return x**3 + x**2 - 3 * x + 2

In [ ]:
scipy.optimize.minimize(g, x0=10)

In [ ]:
x = np.linspace(-3, 2, 100)
px.line(x=x, y=g(x))

In [ ]:
scipy.optimize.minimize(mse_given_theta, x0=0.2).x

In [ ]:
# Minimizer can fail!
scipy.optimize.minimize(g, x0=-3)

# 7.8 Absolute and Huber Loss

L1 loss = |y - yhat|

MAE = mean absolute error = mean of L1 loss over data set

Choice of loss function matters! For example if you minimize L2 you don't necessarily minimize L1, and vice versa

L2 more heavily penalizes outliers than L1. Since L2 more affected by outliers, its solution also more affected by outliers, i.e. your solution is influenced more heavily by outliers

In [ ]:
4.11**2

Showing plot of MAE vs. MSE over the tips data set

![alt text](./data/MAE_vs_MSE.png "MAE vs. MSE")

Note that MAE is piecewise linear, one impact of that is that its derivative is discontinous, whereas the derivative of MSE is continous. That means MAE not as well suited to numerical methods

## Huber Loss

A third loss function mentioned is the Huber loss function.  This is notable for its resistance to extreme values and is defined as a piecewise function:


$${\displaystyle L_{\delta }(y,f(x))={\begin{cases}{\frac {1}{2}}(y-f(x))^{2}&{\textrm {for}}|y-f(x)|\leq \delta ,\\\delta \,(|y-f(x)|-{\frac {1}{2}}\delta ),&{\textrm {otherwise.}}\end{cases}}}$$



# 7.9 Multiple Linear Regression



## Define a 2D fitting object

In [ ]:
features = data[["total_bill", "size"]]
tip = data["tip"]
f2 = linear_model.LinearRegression(fit_intercept=False)
f2.fit(features, tip)

## Look at its Coefs

In [ ]:
f2.coef_

## Make a Prediction

In [ ]:
f2.predict([[10, 3]])

## Evaluate Coefs on All Features

In [ ]:
data["prediction_2d"] = f2.predict(features)

In [ ]:
data

## Compare MSE of 1 and 2D Fits

In [ ]:
[
    mean_squared_error(data["tip"], data["prediction"]),
    mean_squared_error(data["tip"], data["prediction_2d"]),
]

## 3d Scatter Plot

In [ ]:
px.scatter_3d(data, x="total_bill", y="size", z="tip")

## 3d Scatter Plot with Plane from Model Fit

In [ ]:
(table_bills, table_sizes) = np.meshgrid(range(50), range(6))
predictions_mesh = f2.coef_[0] * table_bills + f2.coef_[1] * table_sizes

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter3d(
        x=data["total_bill"],
        y=data["size"],
        z=data["tip"],
        mode="markers",
        name="actual",
    )
)

fig.add_trace(
    go.Surface(
        x=table_bills,
        y=table_sizes,
        z=predictions_mesh,
        name="predicted",
    )
)


fig.show()